In [142]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

import netCDF4
import glob

In [143]:
# turbines and sector power output
turbines = pd.read_csv('windturbines_fillna_shiftlon_xy.csv')
turbines.drop(columns=['Unnamed: 0.1'], inplace=True)

In [144]:
turbine_features = ['Price region', 'Installation date']

In [145]:
#create subset dataframes for each sector
turbines_SE1 = turbines.loc[turbines['Price region']=='SE1', turbine_features]
turbines_SE1.reset_index(inplace=True)
turbines_SE2 = turbines.loc[turbines['Price region']=='SE2', turbine_features]
turbines_SE2.reset_index(inplace=True)
turbines_SE3 = turbines.loc[turbines['Price region']=='SE3', turbine_features]
turbines_SE3.reset_index(inplace=True)
turbines_SE4 = turbines.loc[turbines['Price region']=='SE4', turbine_features]
turbines_SE4.reset_index(inplace=True)

In [146]:
listoffiles=(glob.glob("data/*.nc"))

In [147]:
#Extract timestamps from all the weather files as a list
    
timestamp_list = []

for file in listoffiles:
    weather = netCDF4.Dataset(file, 'r')
    var = weather.variables['time'][:]
    units = weather.variables['time'].units
    Timestamp = netCDF4.num2date(var, units=units)
    timestamp_list.append(Timestamp)

timestamp_list_reordered = []
for i in range(len(timestamp_list)):
    for j in range(len(timestamp_list[i])):
        a = timestamp_list[i][j]
        timestamp_list_reordered.append(a)

In [150]:
timestamp_list_reordered[0] 

cftime.DatetimeGregorian(2000-01-01 00:00:00)

In [169]:
#Extract the string of interest to form the date
list_dates = []
for date in timestamp_list_reordered:
    list_dates.append(timestamp_list_reordered[i].strftime("%Y-%m-%d"))  #cftime date format to string
    

timestamps_formatted = []
for date in list_dates:
    datetime_obj = datetime.datetime.strptime(date, '%Y-%m-%d') #string to date again, now in python datetime.datetime format
    a = datetime_obj.date() #datetime.date(YYYY,mm,dd)
    timestamps_formatted.append(a)

In [167]:
#Transform the installation date string to a datetime.date()

list_SE1 = []
for date in turbines_SE1['Installation date'].values:
    a = datetime.datetime.strptime(date, "%d/%m/%y").strftime('%Y-%m-%d') #change format from dd/mm/yy to YYYY-mm-dd
    b = datetime.datetime.strptime(a, format_str) 
    b.date()
    if b > datetime.datetime.now():
        b = b - relativedelta(years=100) #to avoid years 2050 etc
    list_SE1.append(b)

In [168]:
#Some turbines have installation date 1881-12-31: either we modify it here or it is handled as an exception in the next cell
turbines_SE2.loc[turbines_SE2['Installation date']=='1881-12-31', 'Installation date'] = '31/12/50'
turbines_SE3.loc[turbines_SE3['Installation date']=='1881-12-31', 'Installation date'] = '31/12/50'
turbines_SE4.loc[turbines_SE4['Installation date']=='1881-12-31', 'Installation date'] = '31/12/50'

In [135]:
list_SE2 = []
for date in turbines_SE2['Installation date'].values:
    a = datetime.datetime.strptime(date, "%d/%m/%y").strftime('%Y-%m-%d') 
    b = datetime.datetime.strptime(a, format_str)
    b.date()
    if b > datetime.datetime.now():
        b = (b - relativedelta(years=100))
    list_SE2.append(b)
turbines_SE2['date'] = list_SE2
    
list_SE3 = []
for date in turbines_SE3['Installation date'].values:
    a = datetime.datetime.strptime(date, "%d/%m/%y").strftime('%Y-%m-%d')
    b = datetime.datetime.strptime(a, format_str)
    b.date()
    if b > datetime.datetime.now():
        b = (b - relativedelta(years=100))
    list_SE3.append(b)
turbines_SE3['date'] = list_SE3

list_SE4 = []
for date in turbines_SE4['Installation date'].values:
    a = datetime.datetime.strptime(date, "%d/%m/%y").strftime('%Y-%m-%d')
    b = datetime.datetime.strptime(a, format_str)
    b.date()
    if b > datetime.datetime.now():
        b = (b - relativedelta(years=100))
    list_SE4.append(b)
turbines_SE4['date'] = list_SE4

In [140]:
#quick check to see if the formats are ok
turbines_SE2.head()

,index,Terrain height [m],Nacelle height [m],Rotor diameter [m],Max power [MW],Price region,Installation date,Longitude,Latitude,x,y,"(y,x)",date
0,3,413.00,98.0,90.0,0.02,SE2,31/12/50,13.376852,63.437566,24,93,"(93, 24)",1950-12-31
1,4,413.58,98.0,90.0,0.10,SE2,31/12/50,13.376548,63.436982,24,93,"(93, 24)",1950-12-31
2,5,79.22,17.0,11.0,0.01,SE2,31/12/50,19.174957,63.523244,53,94,"(94, 53)",1950-12-31
3,107,176.25,40.0,44.0,0.60,SE2,01/05/78,17.952315,62.609209,47,84,"(84, 47)",1978-05-01
4,151,100.04,40.0,44.0,0.60,SE2,27/10/78,18.059759,62.755146,48,86,"(86, 48)",1978-10-27


In [170]:
installed_SE1 = np.zeros(shape=(len(timestamp_list_reordered),len(list_SE1)))
installed_SE2 = np.zeros(shape=(len(timestamp_list_reordered),len(list_SE2)))
installed_SE3 = np.zeros(shape=(len(timestamp_list_reordered),len(list_SE3)))
installed_SE4 = np.zeros(shape=(len(timestamp_list_reordered),len(list_SE4)))

for i, date1 in enumerate(timestamps_formatted):
    for j, date2 in enumerate(list_SE1):
        installed_SE1[i][j] = int(date1>date2.date())
        
for i, date1 in enumerate(timestamps_formatted):
    for j, date2 in enumerate(list_SE2):
        installed_SE2[i][j] = int(date1>date2.date())
        
for i, date1 in enumerate(timestamps_formatted):
    for j, date2 in enumerate(list_SE3):
        installed_SE3[i][j] = int(date1>date2.date())
        
for i, date1 in enumerate(timestamps_formatted):
    for j, date2 in enumerate(list_SE4):
        installed_SE4[i][j] = int(date1>date2.date())


In [172]:
installed_SE2

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [173]:
installed_SE2.shape

(10152, 1112)

In [179]:
installed_SE2[10151,1090]

0.0

In [178]:
turbines_SE2.head(-20)

,index,Price region,Installation date
0,3,SE2,31/12/50
1,4,SE2,31/12/50
2,5,SE2,31/12/50
3,107,SE2,01/05/78
4,151,SE2,27/10/78
...,...,...,...
1087,3935,SE2,31/10/01
1088,3937,SE2,31/10/01
1089,3939,SE2,31/10/01
1090,3940,SE2,31/10/01
